# Лабораторная работа №5 Проведение исследований с градиентным бустингом

Подключим базовые модули для выполнения лабораторной работы

In [1]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

Загрузим датасет

In [2]:
kagglehub.dataset_download("yasserh/wine-quality-dataset")
kagglehub.dataset_download("shree1992/housedata")

100%|██████████| 21.5k/21.5k [00:00<00:00, 25.0MB/s]

Extracting files...


100%|██████████| 432k/432k [00:00<00:00, 1.08MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2'

Используем функции из прошлой лабораторной работы для загрузки датасета

In [3]:
def prepare_train_test_split_class(dataset_path, test_size=0.2, random_state=7575):
    df = pd.read_csv(dataset_path)

    if 'Id' in df.columns:
        df = df.drop(columns=['Id'])

    y = df['quality']
    X = df.drop(columns=['quality'])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True
    )

    return X_train, X_test, y_train, y_test

In [4]:
def prepare_train_test_split_reg(dataset_path, test_size=0.2, random_state=7575):
    df = pd.read_csv(dataset_path)

    drop_columns = ['street', 'country', 'date']
    df = df.drop(columns=[col for col in drop_columns if col in df.columns])

    categorical_features = ['city', 'statezip']
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(df[categorical_features])
    encoded_cols = encoder.get_feature_names_out(categorical_features)
    encoded_df = pd.DataFrame(encoder.transform(df[categorical_features]), columns=encoded_cols, index=df.index)
    df = pd.concat([df.drop(columns=categorical_features), encoded_df], axis=1)

    y = df['price']
    X = df.drop(columns=['price'])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True
    )

    return X_train, X_test, y_train, y_test


In [5]:
X_train_class, X_test_class, y_train_class, y_test_class = prepare_train_test_split_class(
    "/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv",
    0.2,
    65366
)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = prepare_train_test_split_reg(
    "/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv",
    0.2,
    653634
)

Импортируем градиетный бустинг из sklearn и обучим

In [6]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

Обучим импортированные модели

In [10]:
grad_class = GradientBoostingClassifier()
grad_class.fit(X_train_class, y_train_class)

GradientBoostingClassifier()

In [11]:
grad_reg = GradientBoostingRegressor()
grad_reg.fit(X_train_reg, y_train_reg)

GradientBoostingRegressor()

Оценим метрики полученных моделей

In [12]:
y_pred_class = grad_class.predict(X_test_class)
y_pred_reg = grad_reg.predict(X_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Gradient Boosting:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Gradient Boosting:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Gradient Boosting:
Accuracy: 0.6332
F1 Score: 0.6255

Regression - Gradient Boosting:
RMSE: 242692.3416
R^2 Score: 0.5567


Как и все более сложные модели градиентный бустинг на сырых данных показал достаточно высокие результаты.

### 3. Улучшение бейзлайна и оценка качества

В данной лабораторной работе я использую бейзлайн и предыдущей, так как он показал достаточно высокий рост метрик и был сформирован в итоге всех лабораторных работ.

Итоговые действия по улучшеню бейзлайна

- Классификация
  1. Убрать признаки residual sugar, pH
  2. Прологарифмируем признаки chlorides и sulphates
  3. Создадим признак-отношение Free Sulfur Dioxide и Total Sulfur Dioxide
- Регрессия
  1. Удалить признак sqft_lot, condition, yr_build, yr_renovated из-за низкой корреляции
  2. Попробуем перевести признак waterfront в бинарный
  3. Обрежем выбросы для bedrooms, bathrooms и price, sqft_living, sqft_above, sqft_basement

### Обучим новые модели на новом бейзлайне

In [13]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['chlorides'] = np.log1p(df_class['chlorides'])
df_class['sulphates'] = np.log1p(df_class['sulphates'])

df_class['sulfur_ratio'] = df_class['free sulfur dioxide'] / df_class['total sulfur dioxide']

y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=5435, shuffle=True
)

In [14]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['yr_built', 'sqft_lot', 'condition', 'yr_renovated', 'country', 'street', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

numerical_features = ['price', 'sqft_living', 'sqft_above', 'sqft_basement', 'bathrooms', 'bedrooms']

for col in numerical_features:
    Q1 = df_reg[col].quantile(0.25)
    Q3 = df_reg[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_reg = df_reg[(df_reg[col] >= lower_bound) & (df_reg[col] <= upper_bound)]

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)


Обучим модели на улучшенном бейзлайне

In [15]:
grad_class_improve = GradientBoostingClassifier()
grad_class_improve.fit(X_train_class, y_train_class)

GradientBoostingClassifier()

In [16]:
grad_reg_improve= GradientBoostingRegressor()
grad_reg_improve.fit(X_train_reg, y_train_reg)

GradientBoostingRegressor()

Оценим метрики получившихся моделей

In [18]:
y_pred_class = grad_class_improve.predict(X_test_class)
y_pred_reg = grad_reg_improve.predict(X_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Gradient Boosting Improve:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Gradient Boosting Improve:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Gradient Boosting Improve:
Accuracy: 0.6594
F1 Score: 0.6543

Regression - Gradient Boosting Improve:
RMSE: 117536.6868
R^2 Score: 0.6916


Бейзлайн довольно сильной повысил точность модели. В задаче регрессии RMSE упало в 2 раза

### Сравнение бейзлайнов

- Классификация

|          | Первый baseline | Улучшенный baseline |
|----------|-----------------|---------------------|
| Accuracy | 0.6332          | 0.6594              |
| F1       | 0.6255          | 0.6543              |

- Регрессия

|       | Первый baseline | Улучшенный baseline |
|-------|-----------------|---------------------|
| RMSE  | 242692.34       | 117536.68           |
| $R^2$ | 0.5567          | 0.6916              |

Метрики достаточно сильно выросли по сравнению с сырым датасетом. Градиентный бустинг очень хорошо показал себя в задаче регрессии и является одним из лучших в решении данной задачи

## 4. Имплементация алгоритма машинного обучения

Теперь реализуем свой алгоритм градиентного бустинга. Как и в прошлых лабораторных работах реализуем в виде универсального класса, который будем параметризовать для конкретной задачи

In [48]:
class DecisionTreeModel:
    def __init__(self, mode="classification", max_levels=None):
        self.mode = mode
        self.max_levels = max_levels
        self.root = None

    def train(self, features, labels):
        self.root = self._grow_tree(features, labels, current_depth=0)

    def predict(self, features):
        return np.array([self._navigate_tree(sample, self.root) for sample in features])

    def _grow_tree(self, features, labels, current_depth):
        if self.max_levels is not None and current_depth >= self.max_levels:
            return self._create_terminal_node(labels)

        if len(np.unique(labels)) == 1:
            return self._create_terminal_node(labels)

        split_feature, split_value = self._identify_optimal_split(features, labels)
        if split_feature is None:
            return self._create_terminal_node(labels)

        left_indices = features[:, split_feature] <= split_value
        right_indices = features[:, split_feature] > split_value

        left_branch = self._grow_tree(features[left_indices], labels[left_indices], current_depth + 1)
        right_branch = self._grow_tree(features[right_indices], labels[right_indices], current_depth + 1)

        return {
            "split_feature": split_feature,
            "split_value": split_value,
            "left": left_branch,
            "right": right_branch
        }

    def _identify_optimal_split(self, features, labels):
        optimal_feature, optimal_value = None, None
        best_metric = float("inf") if self.mode == "regression" else -float("inf")

        for feature_idx in range(features.shape[1]):
            unique_values = np.unique(features[:, feature_idx])
            for value in unique_values:
                left_mask = features[:, feature_idx] <= value
                right_mask = features[:, feature_idx] > value

                if sum(left_mask) == 0 or sum(right_mask) == 0:
                    continue

                metric = self._evaluate_split(labels, left_mask, right_mask)

                if (self.mode == "regression" and metric < best_metric) or (
                    self.mode == "classification" and metric > best_metric
                ):
                    best_metric = metric
                    optimal_feature = feature_idx
                    optimal_value = value

        return optimal_feature, optimal_value

    def _evaluate_split(self, labels, left_mask, right_mask):
        if self.mode == "regression":
            left_error = np.var(labels[left_mask]) * sum(left_mask)
            right_error = np.var(labels[right_mask]) * sum(right_mask)
            return left_error + right_error
        else:
            left_score = self._compute_gini(labels[left_mask]) * sum(left_mask)
            right_score = self._compute_gini(labels[right_mask]) * sum(right_mask)
            return -(left_score + right_score)

    def _compute_gini(self, labels):
        proportions = np.bincount(labels.astype(int)) / len(labels)
        return 1 - np.sum(proportions ** 2)

    def _create_terminal_node(self, labels):
        if self.mode == "regression":
            return np.mean(labels)
        else:
            return np.bincount(labels.astype(int)).argmax()

    def _navigate_tree(self, sample, node):
        if not isinstance(node, dict):
            return node

        if sample[node["split_feature"]] <= node["split_value"]:
            return self._navigate_tree(sample, node["left"])
        else:
            return self._navigate_tree(sample, node["right"])

class GradientBoosting:
    def __init__(self, mode="classification", n_estimators=100, learning_rate=0.1, max_levels=None):
        self.mode = mode
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_levels = max_levels
        self.models = []
        self.initial_prediction = None

    def _initialize_model(self):
        return DecisionTreeModel(mode="regression", max_levels=self.max_levels)

    def _initialize_predictions(self, labels):
        if self.mode == "regression":
            return np.mean(labels)
        else:
            class_counts = np.bincount(labels)
            probabilities = class_counts / len(labels)
            return np.log(probabilities + 1e-10)

    def _compute_residuals(self, labels, predictions):
        if self.mode == "regression":
            return labels - predictions
        else:
            if len(predictions.shape) == 1:
                predictions = predictions.reshape(-1, 1)
            probabilities = self._softmax(predictions)
            residuals = np.eye(np.max(labels) + 1)[labels] - probabilities
            return residuals

    def _softmax(self, logits):
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

    def train(self, features, labels):
        self.models = []

        if self.mode == "classification":
            unique_labels = np.unique(labels)
            self.initial_prediction = self._initialize_predictions(labels)
            predictions = np.tile(self.initial_prediction, (features.shape[0], 1))
        else:
            self.initial_prediction = self._initialize_predictions(labels)
            predictions = np.full(len(labels), self.initial_prediction)

        for _ in range(self.n_estimators):
            residuals = self._compute_residuals(labels, predictions)

            if self.mode == "classification":
                model = [self._initialize_model() for _ in range(residuals.shape[1])]
                for class_idx, tree in enumerate(model):
                    tree.train(features, residuals[:, class_idx])
                self.models.append(model)
                updates = np.array([
                    tree.predict(features) for tree in model
                ]).T
            else:
                model = self._initialize_model()
                model.train(features, residuals)
                self.models.append(model)
                updates = model.predict(features)

            predictions += self.learning_rate * updates

    def predict(self, features):
        if self.mode == "classification":
            predictions = np.tile(self.initial_prediction, (features.shape[0], 1))
            for model in self.models:
                updates = np.array([
                    tree.predict(features) for tree in model
                ]).T
                predictions += self.learning_rate * updates
            return np.argmax(self._softmax(predictions), axis=1)
        else:
            predictions = np.full(features.shape[0], self.initial_prediction)
            for model in self.models:
                predictions += self.learning_rate * model.predict(features)
            return predictions

Попробуем обучить реализованную модель на сыром датасете

In [49]:
X_train_class, X_test_class, y_train_class, y_test_class = prepare_train_test_split_class(
    "/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv",
    0.2,
    65366
)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = prepare_train_test_split_reg(
    "/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv",
    0.2,
    653634
)

In [52]:
grad_class_impl = GradientBoosting(n_estimators=20, learning_rate=0.1, max_levels=3, mode="classification")
grad_class_impl.train(X_train_class.to_numpy(), y_train_class.to_numpy())

In [56]:
grad_reg_impl = GradientBoosting(n_estimators=20, learning_rate=0.1, max_levels=3, mode="regression")
grad_reg_impl.train(X_train_reg.to_numpy(), y_train_reg.to_numpy())

In [57]:
y_pred_class = grad_class_impl.predict(X_test_class.to_numpy())
y_pred_reg = grad_reg_impl.predict(X_test_reg.to_numpy())

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Gradient Boosting Implementation:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Gradient Boosting Implementation:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Gradient Boosting Implementation:
Accuracy: 0.5284
F1 Score: 0.4830

Regression - Gradient Boosting Implementation:
RMSE: 257931.1389
R^2 Score: 0.4993


Как можно заметить реализация градиентного бустинга показала себя чуть хуже в задаче классификации, чем остальные реализации. В то же время задача регрессии показала достаточно хорошие результаты даже на базовом бейзлайне

### Сравнение с реализацией в библиотеке

- Классификация

|          | Библиотека      | Имплементация       |
|----------|-----------------|---------------------|
| Accuracy | 0.6332          | 0.5284              |
| F1       | 0.6255          | 0.4830              |

- Регрессия

|        | Библиотека      | Имплиментация    |
|--------|-----------------|------------------|
| RMSE   | 242692.34       | 257931.13        |
| $R^2$  | 0.5567          | 0.4993           |

Как можно заметить впервые реализация намного уступает библиотеке. Скорее всего это связано с различными подходами к реализации алгоритмов. Существует большое количество вариаций градиентного бустинга, такие как CatBoost или XGDBoost

### Обучим модель на улучшенном бейзлайне

Подготовим улучшенный бейзлайн

In [58]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['chlorides'] = np.log1p(df_class['chlorides'])
df_class['sulphates'] = np.log1p(df_class['sulphates'])

df_class['sulfur_ratio'] = df_class['free sulfur dioxide'] / df_class['total sulfur dioxide']

y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=5435, shuffle=True
)

In [59]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['yr_built', 'sqft_lot', 'condition', 'yr_renovated', 'country', 'street', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

numerical_features = ['price', 'sqft_living', 'sqft_above', 'sqft_basement', 'bathrooms', 'bedrooms']

for col in numerical_features:
    Q1 = df_reg[col].quantile(0.25)
    Q3 = df_reg[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_reg = df_reg[(df_reg[col] >= lower_bound) & (df_reg[col] <= upper_bound)]

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)


Обучим модели на улучшенном бейзлайне

In [60]:
grad_class_impl_improve = GradientBoosting(n_estimators=20, learning_rate=0.1, max_levels=3, mode="classification")
grad_class_impl_improve.train(X_train_class.to_numpy(), y_train_class.to_numpy())

In [61]:
grad_reg_impl_improve = GradientBoosting(n_estimators=20, learning_rate=0.1, max_levels=3, mode="regression")
grad_reg_impl_improve.train(X_train_reg.to_numpy(), y_train_reg.to_numpy())

In [62]:
y_pred_class = grad_class_impl_improve.predict(X_test_class.to_numpy())
y_pred_reg = grad_reg_impl_improve.predict(X_test_reg.to_numpy())

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Gradient Boosting Implementation Improve:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Gradient Boosting Implementation Improve:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Gradient Boosting Implementation Improve:
Accuracy: 0.6026
F1 Score: 0.5585

Regression - Gradient Boosting Implementation Improve:
RMSE: 148392.9389
R^2 Score: 0.5084


### Сравнение с реализацией в библиотеке

- Классификация

|          | Библиотека      | Имплементация       |
|----------|-----------------|---------------------|
| Accuracy | 0.6594          | 0.6026              |
| F1       | 0.6543          | 0.5585              |

- Регрессия

|        | Библиотека      | Имплиментация    |
|--------|-----------------|------------------|
| RMSE   | 117536.68       | 148392.93        |
| $R^2$  | 0.6916          | 0.5084           |

Аналогичная ситуация происходит и при применении улучшенного бейзлайна. Но все же ситуация несколько улучшилась

В целом градиетный бустинг показал себя как достаточно сильный метод, но требует долгих исследований, различных переборов параметров и реализаций.

# Итоги

Подведем итоги и выпишем лучшие метрики для каждого алгоритма:

- Классификация

| Алгоритм            | Baseline | Улучшенный бейзлайн | Имплементация | Улучшенный бейзлайн и имплементация |
|---------------------|----------|---------------------|---------------|-------------------------------------|
| KNN                 |Acc: 0.5939 F1: 0.5866|Acc: 0.6375 F1: 0.6224|Acc: 0.5983 F1: 0.5983|Acc: 0.5983 F1: 0.5800|
| Линейные модели     |Acc: 0.5590 F1: 0.5337|Acc: 0.4868 F1: 0.5416|Acc: 0.5458 F1: 0.4806|Acc: 0.4810 F1: 0.5324|
| Решающее дерево     |Acc: 0.5459 F1: 0.5416|Acc: 0.5977 F1: 0.5976|Acc: 0.5371 F1: 0.5248|Acc: 0.5714 F1: 0.5612|
| Случайный лес       |Acc: 0.6900 F1: 0.6801|Acc: 0.6900 F1: 0.6776|Acc: 0.6157 F1: 0.6018|Acc: 0.6201 F1: 0.5991|
| Градиентный бустинг |Acc: 0.6332 F1: 0.6255|Acc: 0.6594 F1: 0.6543|Acc: 0.5284 F1: 0.4830|Acc: 0.6026 F1: 0.5585|

- Регрессия

| Алгоритм            | Baseline | Улучшенный бейзлайн | Имплементация | Улучшенный бейзлайн и имплементация |
|---------------------|----------|---------------------|---------------|-------------------------------------|
| KNN                 |RMSE: 319024.15 $R^2$: 0.2929|RMSE: 113414.29 $R^2$: 0.6902|RMSE: 769682.09 $R^2$: 0.0735|RMSE: 167598.61 $R^2$: 0.3237|
| Линейные модели     |RMSE: 749683.74 $R^2$: 0.1210|RMSE: 211577.87 $R^2$: 0.5442|RMSE: 751253.83 $R^2$: 0.1173|RMSE: 240692.48 $R^2$: 0.4101|
| Решающее дерево     |RMSE: 777331.99 $R^2$: 0.0550|RMSE: 238610.66 $R^2$: 0.4482|RMSE: 761764.25 $R^2$: 0.0925|RMSE: 262317.23 $R^2$: 0.3332|
| Случайный лес       |RMSE: 216876.39 $R^2$: 0.6460|RMSE: 112026.73 $R^2$: 0.7198|RMSE: 262082.23 $R^2$: 0.4830|RMSE: 139889.71 $R^2$: 0.5631|
| Градиентный бустинг |RMSE: 242692.34 $R^2$: 0.5567|RMSE: 117536.68 $R^2$: 0.6916|RMSE: 257931.13 $R^2$: 0.4993|RMSE: 148392.93 $R^2$: 0.5084|

### Лучший в задаче классификации
- Accuracy
  1. Значение: 112026.73
  2. Алгоритм: Случайный лес
  3. Версия: Улучшенный baseline
- F1
  1. Значение: 0.7198
  2. Алгоритм: Случайный лес
  3. Версия: Улучшенный baseline

### Лучший в задаче регрессии
- RMSE
  1. Значение: 0.6900
  2. Алгоритм: Случайный лес
  3. Версия: Улучшенный baseline
- $R^2$
  1. Значение: 0.6801
  2. Алгоритм: Случайный лес
  3. Версия: Обычный baseline

## Вывод

Лучшие результаты во всех задачах показал случайный лес. Неудиветельно, т.к. это ансабль моделей. Также это нелинейная модель, что позволяет еще сильнее повысить точность и работает с нелинейными зависимостями